In [1]:
import pandas as pd 
import numpy as np
import re

In [2]:
# read data from excel
df = pd.read_excel('data.xlsx', skiprows=6)

# print the first 5 rows
df.head()

,Deal ID,Companies,Pending Patent Documents,Active Patent Documents,Total Patent Families,Total Patent Documents,VC Round Up/Down/Flat,Price per Share,% Acquired,Post Valuation Status,...,VC Round,Deal Size,Post Valuation,Deal Type,Keywords,Deal Date,Primary Industry Code,Verticals,HQ Location,View Company Online
0,272893-60T,Wavo (Financial Software),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1st Round,3.63,NaN,Early Stage VC,"capital needs, commercial finance, financial t...",2025-02-17,Financial Software,"Artificial Intelligence & Machine Learning, Fi...","Lyon, France",NaN
1,285153-67T,Ziwig,1.0,6.0,4.0,7.0,NaN,NaN,NaN,NaN,...,3rd Round,NaN,NaN,Later Stage VC,"diagnostic platform, endometriosis treatment, ...",2025-02-14,Discovery Tools (Healthcare),"Artificial Intelligence & Machine Learning, Li...","Lyon, France",NaN
2,285101-92T,Digifood,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3rd Round,NaN,NaN,Later Stage VC,"digital food menu, digital food ordering, food...",2025-02-13,"Other Restaurants, Hotels and Leisure","FoodTech, Mobile, TMT","Vanves, France",NaN
3,285052-87T,Smile Wanted Group,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1st Round,10.37,NaN,Later Stage VC,"brand design, campaign support, campaign work,...",2025-02-13,Media and Information Services (B2B),"AdTech, Marketing Tech","Paris, France",NaN
4,285010-48T,ekstere,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1st Round,2.49,NaN,Later Stage VC,"eco-friendly products online, electric bicycle...",2025-02-12,Automotive,E-Commerce,"Vélizy-Villacoublay, France",NaN


In [3]:
# Build the regex dynamically
ignore_pattern = '|'.join(['Private\s+Equity', 'Financial\s+Services', 'Electrical\s+Equipment'])
pattern = rf'\((?:[A-Z]{2,4}:\s*[A-Z0-9.]+)\)|\((?![A-Z]{2,4}:\s*[A-Z0-9.]+)(?!{ignore_pattern})([A-Za-zÀ-ÖØ-öø-ÿ-]+(?:\s+[A-Za-zÀ-ÖØ-öø-ÿ-]+)+)\)'

# Compile regex for better performance
compiled_regex = re.compile(pattern)


In [4]:
# Select the columns investors
investors = df['Investors']

# Function to extract partner names
def extract_partners(text):
    matches = re.findall(pattern, text)
    # Flatten the list and remove empty strings
    cleaned_matches = [match for sublist in matches for match in sublist if match]
    return ", ".join(cleaned_matches) if cleaned_matches else None
    

# Apply function to create 'partners' column only to the first 5 rows
df["Partners"] = investors.apply(extract_partners)

In [5]:
df['Partners'].head(20)

0                                           Tony Parker
1                                                  None
2                                                  None
3                        Ménelé Chesnot, Benjamin Cohen
4                                                  None
5                                        Arnaud Vincent
6                                          Bruno Akpaka
7                                                  None
8                                                  None
9                                                  None
10    Herve Silbert, Anne-Sophie Saint-Martin, Rapha...
11                                                 None
12                                                 None
13                                                 None
14                         Marcell Vollmer, Mario Götze
15                                                 None
16                                                 None
17    Charles Gorintin, Marie Brayer, Jean-Louis

In [6]:
from collections import Counter

# Function to filter two-word names
def get_two_word_names(partners_list):
    two_word_names = []
    for names in partners_list:
        if names:  # Ensure it's not None
            for name in names.split(", "):  # Handle multiple names
                if len(name.split()) == 2:  # Check if it has exactly two words
                    two_word_names.append(name)
    return two_word_names

# Extract all partner names from the column
all_partners = df["Partners"].dropna().tolist()

# Get only two-word names
two_word_names = get_two_word_names(all_partners)

# Count occurrences
name_counts = Counter(two_word_names)

# Convert to DataFrame for visualization
df_name_counts = pd.DataFrame(name_counts.items(), columns=["Two-Word Name", "Frequency"]).sort_values(by="Frequency", ascending=False)

# Display the frequency table
df_name_counts.head(50)


,Two-Word Name,Frequency
438,Xavier Niel,99
2213,Guillaume Aubin,41
250,Thibaud Elziere,41
914,Joséphine Marie,33
842,Jean-David Chamboredon,28
38,Philippe Collombel,25
1425,Chahra Louafi,24
1218,Romain Lavault,24
788,Rodolphe Menegaux,24
2290,Marc Simoncini,22


In [7]:
# Drop rows where 'partners' column is NaN
df = df.dropna(subset=['Partners'])

# Verify the result
df.head()

,Deal ID,Companies,Pending Patent Documents,Active Patent Documents,Total Patent Families,Total Patent Documents,VC Round Up/Down/Flat,Price per Share,% Acquired,Post Valuation Status,...,Deal Size,Post Valuation,Deal Type,Keywords,Deal Date,Primary Industry Code,Verticals,HQ Location,View Company Online,Partners
0,272893-60T,Wavo (Financial Software),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.63,NaN,Early Stage VC,"capital needs, commercial finance, financial t...",2025-02-17,Financial Software,"Artificial Intelligence & Machine Learning, Fi...","Lyon, France",NaN,Tony Parker
3,285052-87T,Smile Wanted Group,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,10.37,NaN,Later Stage VC,"brand design, campaign support, campaign work,...",2025-02-13,Media and Information Services (B2B),"AdTech, Marketing Tech","Paris, France",NaN,"Ménelé Chesnot, Benjamin Cohen"
5,284916-61T,Germitec,14.0,78.0,18.0,103.0,NaN,NaN,NaN,NaN,...,30.05,NaN,Later Stage VC,"disinfectants care, disinfection product, disi...",2025-02-12,Monitoring Equipment,HealthTech,"Bordeaux, France",NaN,Arnaud Vincent
6,279892-63T,ToumAI Analytics,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.04,NaN,Early Stage VC,"customer relationship management, multichannel...",2025-02-10,Business/Productivity Software,"Artificial Intelligence & Machine Learning, SaaS","Cannes, France",NaN,Bruno Akpaka
10,284336-56T,Reev (Therapeutic Devices),1.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,...,9.20,NaN,Early Stage VC,"healthcare service, muscle strengthening tools...",2025-02-06,Therapeutic Devices,HealthTech,"Toulouse, France",NaN,"Herve Silbert, Anne-Sophie Saint-Martin, Rapha..."


In [ ]:
# Save the results to a new csv file
df.to_csv('data.csv', index=False)

: 